In [ ]:
%cd /content/drive/MyDrive

In [ ]:
!git clone https://github.com/clovaai/deep-text-recognition-benchmark.git

In [ ]:
!gdown 1b59rXuGGmKne1AuHnkgDzoYgKeETNMv9

In [ ]:
%cd deep-text-recognition-benchmark

In [ ]:
!unzip -qq plate_img-test.zip

In [ ]:
!pip install lmdb
!pip3 install fire

In [ ]:
import xml.etree.ElementTree as ET
import os

def is_latin_char(character):
    return character.isalpha() and ord(character) < 128

def is_latin_numeral(character):
    return character.isdigit() and ord(character) < 128

dataclass = ['train','validation']
dicts = {
        'B': 'ب',
        'P': 'پ',
        'T': 'ت',
        'Y': 'ث',
        'Z': 'ز',
        'X': 'ش',
        'E': 'ع',
        'F': 'ف',
        'K': 'ک',
        'G': 'گ',
        'D': 'D',
        'S': 'S',
        'J': 'ج',
        'W': 'د',
        'C': 'س',
        'U': 'ص',
        'R': 'ط',
        'Q': 'ق',
        'L': 'ل',
        'M': 'م',
        'N': 'ن',
        'V': 'و',
        'H': 'ه',
        'I': 'ی',
        '0': '۰',
        '1': '۱',
        '2': '۲',
        '3': '۳',
        '4': '۴',
        '5': '۵',
        '6': '۶',
        '7': '۷',
        '8': '۸',
        '9': '۹',

}
Move = {
        'A': 'الف',
        '@': 'ویلچر',
    }
for i in dataclass:
    output_file_path = f'gt_{i}.txt'
    f = open(output_file_path, 'w')
    for file in os.listdir(f'{i}/'):
        error_file = ''
        if file.endswith('.xml'):
            error_file = file
            tree = ET.parse(f'{i}/{file}')
            root = tree.getroot()
            filename = root.find('filename').text
            print(f'Filename: {file}')
            name = ''
            name_new=''
            for obj in root.findall('object'):
                name += obj.find('name').text
            # print(name)
            for key in Move:
                if name.find(Move[key]) != -1:
                    name_new=name.replace(Move[key], key)
                    name = ''
                    name = name_new
            name_new=''
            for index, v in enumerate(name):
                if is_latin_char(v)==False and is_latin_numeral(v) == False:
                    for key in dicts:
                        if dicts[key] == v:
                            new_name=name.replace(name[index], key)
                            name = ''
                            name = new_name
            filejpg = file.split('.')
            filesave = filejpg[0] + ".jpg"
            f.write(f'{filesave}	{name}\n')

In [ ]:
!python3 create_lmdb_dataset.py --inputPath train/ --gtFile gt_train.txt --outputPath result/train

In [ ]:
!python3 create_lmdb_dataset.py --inputPath validation/ --gtFile gt_validation.txt --outputPath result/validation

In [ ]:
!python3 train.py \
--train_data result/train --valid_data result/validation \
--select_data / --batch_ratio 1 --batch_max_length 8 --valInterval 100 --num_iter 4900 \
--Transformation TPS --FeatureExtraction ResNet --SequenceModeling BiLSTM --Prediction Attn

In [ ]:
!python3 create_lmdb_dataset.py --inputPath test/ --gtFile gt_test.txt --outputPath result/test

In [ ]:
!python3 test.py \
--eval_data result/test \
--saved_model saved_models/TPS-ResNet-BiLSTM-Attn-Seed1111/best_accuracy.pth \
--Transformation TPS --FeatureExtraction ResNet --SequenceModeling BiLSTM --Prediction Attn

In [ ]:
!python3 demo.py \
--Transformation TPS --FeatureExtraction ResNet --SequenceModeling BiLSTM --Prediction Attn \
--image_folder test_imag --batch_max_length 8 \
--saved_model saved_models/TPS-ResNet-BiLSTM-Attn-Seed1111/best_accuracy.pth

In [ ]:
Train:
73.918
Test:
73.998
